In [1]:
import sys
print(sys.version)

3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]


In [1]:
import opendatasets as od

In [2]:
readability_dataset_url = "https://www.kaggle.com/datasets/karthikudyawar/readability"

In [4]:
od.download(readability_dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading readability.zip to .\readability


100%|██████████| 11.4k/11.4k [00:00<00:00, 4.01MB/s]